# Lab 4 — Speech Emotion Recognition (SER)

Notebook này triển khai pipeline đầy đủ cho SER sử dụng Python, Librosa và TensorFlow/Keras. Bạn có thể chạy trên Google Colab (khuyến nghị) hoặc máy cá nhân có GPU.


## Mục tiêu & Deliverables

- Tiền xử lý audio (chuẩn hóa, cắt/pad, tăng cường dữ liệu)
- Trích xuất đặc trưng (log-mel spectrogram, MFCC + delta)
- Huấn luyện mô hình CNN + BiLSTM phân loại cảm xúc
- Đánh giá: accuracy, classification report, confusion matrix
- Suy luận (inference) với file `.wav` của bạn
- Xuất model `.h5` và lưu các biểu đồ loss/accuracy


In [ ]:
# Nếu chạy trên Colab, nên bật GPU: Runtime -> Change runtime type -> GPU
import librosa.display
import librosa
from tensorflow.keras import layers, models, callbacks, optimizers
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import soundfile as sf
import sys
import os
import platform
import subprocess
import pathlib
import math
import json
import random
import time
import zipfile
import shutil
import numpy as np
import pandas as pd

# Cài đặt thư viện cần thiết


def pip_install(pkg):
    try:
        __import__(pkg.split("==")[0].split(">=")[0].split("<=")[0])
    except ImportError:
        print(f"Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])


for pkg in [
    "librosa>=0.10.1",
    "soundfile",
    "matplotlib",
    "scikit-learn",
    "tensorflow>=2.13",
    "tqdm",
]:
    pip_install(pkg)

print("TensorFlow:", tf.__version__)


## Dữ liệu

Notebook hỗ trợ **RAVDESS** và **EmoDB**. Do vấn đề bản quyền/phân phối, bạn tự chuẩn bị dữ liệu.

**Tùy chọn A (khuyên dùng):** tải dữ liệu về Google Drive rồi chỉ đường dẫn.

**Tùy chọn B (demo nhanh):** đặt một thư mục `datasets/sample` chứa vài file `.wav` theo cấu trúc:

```
datasets/
  ravdess/Audio_Speech_Actors_01-24/... (các file .wav)
  emodb/wav/... (các file .wav)
```


In [ ]:
# Định cấu hình đường dẫn dữ liệu ở đây
DATA_DIRS = {
    "ravdess": "/content/datasets/ravdess",  # thay bằng đường dẫn của bạn
    "emodb":   "/content/datasets/emodb",    # thay bằng đường dẫn của bạn
}

# Nếu bạn dùng Jupyter local, có thể thay /content bằng đường dẫn máy bạn.
# Với Colab + Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# DATA_DIRS["ravdess"] = "/content/drive/MyDrive/datasets/ravdess"
# DATA_DIRS["emodb"] = "/content/drive/MyDrive/datasets/emodb"

for k, v in DATA_DIRS.items():
    print(f"{k}: {v} -> {'OK' if os.path.isdir(v) else 'MISSING'}")


### Mapping nhãn cảm xúc

RAVDESS mã hóa cảm xúc trong tên file (thẻ `-XX-`), EmoDB dùng ký hiệu chữ cái. Bảng mapping sau chuẩn hóa về 7 lớp:
`neutral, calm, happy, sad, angry, fearful, disgust` (có thể không đủ trên mọi bộ dữ liệu; ta sẽ tự động gộp khi thiếu).


In [ ]:
RAVDESS_EMO_MAP = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"  # có thể loại bỏ nếu không muốn
}

# EmoDB: (Berlin) codes in filename like 'xxxyy.wav' with emotion letter at position 5
EMODB_MAP = {
    "W": "angry",
    "L": "boredom",   # có thể gộp vào 'neutral'
    "E": "disgust",
    "A": "fearful",
    "F": "happy",
    "T": "sad",
    "N": "neutral"
}

TARGET_LABELS = ["neutral", "calm", "happy",
                 "sad", "angry", "fearful", "disgust"]
CANONICAL = {  # gộp label ngoài chuẩn vào nhãn gần nhất
    "boredom": "neutral",
    "surprised": "happy"  # thay đổi nếu bạn muốn giữ 'surprised' riêng
}


In [ ]:
import re
import glob


def parse_emotion_from_ravdess(path):
    # RAVDESS: filename like '03-01-05-01-02-01-12.wav' -> the 3rd group is emotion code
    m = re.search(r"(\d+)-(\d+)-(\d+)-", os.path.basename(path))
    if not m:
        return None
    emo_code = m.group(3)
    return RAVDESS_EMO_MAP.get(emo_code)


def parse_emotion_from_emodb(path):
    # EmoDB: filename like '03a01Fa.wav' -> letter at position 6 is emotion
    base = os.path.basename(path)
    m = re.search(r"([A-Za-z])\d?\.wav$", base)
    # Safer parse: find the first capital letter among {W,L,E,A,F,T,N}
    for ch in base:
        if ch.upper() in EMODB_MAP:
            return EMODB_MAP[ch.upper()]
    return None


def canonicalize(label):
    return CANONICAL.get(label, label)


rows = []
for ds, root in DATA_DIRS.items():
    if not os.path.isdir(root):
        continue
    pattern = "**/*.wav"
    for p in glob.glob(os.path.join(root, pattern), recursive=True):
        emo = None
        if ds == "ravdess":
            emo = parse_emotion_from_ravdess(p)
        elif ds == "emodb":
            emo = parse_emotion_from_emodb(p)
        if emo:
            rows.append({"path": p, "dataset": ds,
                        "emotion_raw": emo, "emotion": canonicalize(emo)})

df = pd.DataFrame(rows)
print("Total files:", len(df))
print(df.emotion.value_counts())
df.head()


## Trích xuất đặc trưng

Ta dùng **log-mel spectrogram** hình ảnh thời gian–tần số cố định để huấn luyện CNN+BiLSTM. Thêm MFCC + delta làm kênh phụ (tùy chọn).


In [ ]:
SR = 16000            # sample rate
DURATION = 3.0        # giây, mỗi mẫu sẽ pad/trim về cố định
N_MELS = 64
N_FFT = 1024
HOP_LEN = 256


def load_wav(path, sr=SR, duration=DURATION):
    y, _ = librosa.load(path, sr=sr, mono=True)
    target_len = int(sr * duration)
    if len(y) < target_len:
        y = np.pad(y, (0, target_len - len(y)))
    else:
        y = y[:target_len]
    return y


def extract_features(y, sr=SR):
    mel = librosa.feature.melspectrogram(
        y=y, sr=sr, n_fft=N_FFT, hop_length=HOP_LEN, n_mels=N_MELS)
    logmel = librosa.power_to_db(mel, ref=np.max)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    d_mfcc = librosa.feature.delta(mfcc)
    dd_mfcc = librosa.feature.delta(mfcc, order=2)
    # Chuẩn hóa theo trục thời gian

    def norm(x):
        mu = np.mean(x, axis=1, keepdims=True)
        sd = np.std(x, axis=1, keepdims=True) + 1e-9
        return (x - mu)/sd
    return norm(logmel), norm(mfcc), norm(d_mfcc), norm(dd_mfcc)


# Demo 1 file nếu có
if len(df) > 0:
    y = load_wav(df.iloc[0]["path"])
    logmel, *_ = extract_features(y)
    plt.figure(figsize=(8, 3))
    librosa.display.specshow(
        logmel, sr=SR, hop_length=HOP_LEN, x_axis="time", y_axis="mel")
    plt.title("Log-mel spectrogram (demo)")
    plt.colorbar(format="%+2.0f dB")
    plt.show()


In [ ]:
def build_arrays(frame: pd.DataFrame):
    X_mel, X_mfcc, X_d1, X_d2, y = [], [], [], [], []
    for _, r in tqdm(frame.iterrows(), total=len(frame)):
        try:
            y_wav = load_wav(r["path"])
            mel, mfcc, d1, d2 = extract_features(y_wav)
            # Đưa về shape (T, F, C)
            mel = mel.T[:, :, None]
            # Có thể concatenate MFCC/deltas theo kênh nếu muốn
            X_mel.append(mel.astype(np.float32))
            y.append(r["emotion"])
        except Exception as e:
            print("Skip", r["path"], e)
    X = np.stack(X_mel)
    label2id = {lab: i for i, lab in enumerate(sorted(frame.emotion.unique()))}
    y_ids = np.array([label2id[v] for v in y])
    id2label = {i: lab for lab, i in label2id.items()}
    return X, y_ids, label2id, id2label


# Lọc chỉ những label có đủ tối thiểu số lượng mẫu
MIN_PER_CLASS = 20
vc = df.emotion.value_counts()
valid_labels = set(vc[vc >= MIN_PER_CLASS].index)
df_f = df[df.emotion.isin(valid_labels)].copy()
print("Giữ", len(valid_labels), "nhãn:",
      valid_labels, " | Số file:", len(df_f))

train_df, test_df = train_test_split(
    df_f, test_size=0.2, random_state=42, stratify=df_f["emotion"])
train_df, val_df = train_test_split(
    train_df, test_size=0.2, random_state=42, stratify=train_df["emotion"])

print("Train/Val/Test:", len(train_df), len(val_df), len(test_df))

X_train, y_train, label2id, id2label = build_arrays(train_df)
X_val, y_val, _, _ = build_arrays(val_df)
X_test, y_test, _, _ = build_arrays(test_df)

num_classes = len(label2id)
input_shape = X_train.shape[1:]  # (T, F, C)
input_shape, num_classes


## Mô hình: CNN + BiLSTM

Kiến trúc gọn nhẹ nhưng hiệu quả cho SER.


In [ ]:
def make_model(input_shape, num_classes):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool2D((2, 2))(x)

    # Collapse frequency dim, keep time dim
    # Shape now: (T/4, F/4, 64)
    t = tf.shape(x)[1]
    f = x.shape[2]
    c = x.shape[3]
    x = layers.Reshape((-1, f*c))(x)  # (T', F'*C)

    x = layers.Bidirectional(layers.LSTM(128, return_sequences=False))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=optimizers.Adam(1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


model = make_model(input_shape, num_classes)
model.summary()


In [ ]:
# Class weights để xử lý mất cân bằng
from sklearn.utils.class_weight import compute_class_weight

classes = np.arange(num_classes)
class_weights = compute_class_weight(class_weight="balanced",
                                     classes=classes,
                                     y=y_train)
class_weights = {i: w for i, w in enumerate(class_weights)}
print("class_weights:", class_weights)

ckpt_path = "best_model.keras"
cbs = [
    callbacks.ModelCheckpoint(
        ckpt_path, monitor="val_accuracy", save_best_only=True, mode="max"),
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3),
    callbacks.EarlyStopping(
        monitor="val_loss", patience=6, restore_best_weights=True),
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    callbacks=cbs,
    verbose=1
)


In [ ]:
# Vẽ learning curves
plt.figure()
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss curve")
plt.show()

plt.figure()
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Accuracy curve")
plt.show()


In [ ]:
# Đánh giá trên test set
pred = np.argmax(model.predict(X_test), axis=1)
print("Accuracy (test):", np.mean(pred == y_test))

print("\nClassification report:\n")
print(classification_report(y_test, pred, target_names=[
      id2label[i] for i in range(num_classes)]))

cm = confusion_matrix(y_test, pred, labels=list(range(num_classes)))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[
                              id2label[i] for i in range(num_classes)])
fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(ax=ax, xticks_rotation=45)
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()


In [ ]:
# Suy luận với file .wav của bạn
def predict_wav(path):
    y = load_wav(path)
    mel, *_ = extract_features(y)
    x = mel.T[None, :, :, None].astype(np.float32)
    probs = model.predict(x)[0]
    for i, p in enumerate(probs):
        print(f"{id2label[i]:<10}: {p:.3f}")
    print("=> Dự đoán:", id2label[int(np.argmax(probs))])

# Ví dụ:
# predict_wav("/content/some_audio.wav")


In [ ]:
# Lưu model
model.save("ser_cnn_bilstm.h5")
print("Saved ser_cnn_bilstm.h5")


---

### Ghi chú

- Nếu dữ liệu quá ít, hãy giảm `DURATION` hoặc tăng cường dữ liệu.
- Có thể thêm SpecAugment (masking theo thời gian/tần số) để tăng robustness.
- Thử các kiến trúc khác (CNN-only, CRNN sâu hơn, AST/transformer) nếu bạn có GPU mạnh.
